In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("container","raw")
dbutils.widgets.text("catalogo","credit_catalog")
dbutils.widgets.text("esquema","bronze")
dbutils.widgets.text("datalake","adlssmartdata1202")

In [0]:
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")
datalake = dbutils.widgets.get("datalake")

ruta = f"abfss://{container}@{datalake}.dfs.core.windows.net/application_record.csv"

In [0]:
df_application_record = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(ruta)

In [0]:
application_record_schema = StructType(fields=[StructField("ID", IntegerType(), False),
                                     StructField("CODE_GENDER", StringType(), True),
                                     StructField("FLAG_OWN_CAR", StringType(), True),
                                     StructField("FLAG_OWN_REALTY", StringType(), True),
                                     StructField("CNT_CHILDREN", IntegerType(), True),
                                     StructField("AMT_INCOME_TOTAL", DoubleType(), True),
                                     StructField("NAME_INCOME_TYPE", StringType(), True),
                                     StructField("NAME_EDUCATION_TYPE", StringType(), True),
                                     StructField("NAME_FAMILY_STATUS", StringType(), True),
                                     StructField("NAME_HOUSING_TYPE", StringType(), True),
                                     StructField("DAYS_BIRTH", IntegerType(), True),
                                     StructField("DAYS_EMPLOYED", IntegerType(), True),
                                     StructField("OCCUPATION_TYPE", StringType(), True),
                                     StructField("CNT_FAM_MEMBERS", DoubleType(), True)
])

In [0]:
df_application_record_final = spark.read\
.option('header', True)\
.schema(application_record_schema)\
.csv(ruta)

In [0]:
df_application_record_final_date = df_application_record_final.withColumn("INGESTION_DATE", current_timestamp())

In [0]:
df_application_record_final_date.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.application_record")